In [3]:
# --- Hugging Face Login using Kaggle Secrets ---
# Make sure you have set 'HF_TOKEN' in Kaggle Secrets before running this.

from huggingface_hub import login
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_TOKEN")

try:
    login(token=secret_value)
    print("\nSuccessfully logged in to Hugging Face using Kaggle Secret.")
except:
    print("\nError: 'HF_TOKEN' not found in Kaggle Secrets.")
    print("You can manually log in by uncommenting the line below.")


Successfully logged in to Hugging Face using Kaggle Secret.


In [6]:
# مرحله 1: نصب کتابخانه‌های موردنیاز
!pip install transformers accelerate peft bitsandbytes --quiet

# مرحله 2: ایمپورت کتابخانه‌ها
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os

# مرحله 3: بارگذاری مدل پایه
base_model_id = "google/medgemma-4b-it"  # یا هر مدل پایه‌ای که استفاده کردی
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# مرحله 4: بارگذاری توکنایزر
output_dir = "/kaggle/input/farsi-tuned"  # مسیر فایل‌های فاین‌تیون‌شده

# اگر از tokenizer اصلی مدل استفاده کردی
# tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
# ولی اگر tokenizer رو ذخیره کردی و تغییر داده بودی:
tokenizer = AutoTokenizer.from_pretrained(output_dir, trust_remote_code=True)

# مرحله 5: اعمال LoRA روی مدل پایه
model = PeftModel.from_pretrained(model, output_dir)
model.eval()

# مرحله 6: استفاده از مدل
prompt = "علائم اصلی بیماری دیابت چیست؟"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Loading checkpoint shards:  50%|█████     | 1/2 [01:13<01:13, 73.94s/it]


KeyboardInterrupt: 

In [7]:
# --- بارگذاری مدل فاین تیون شده در نوت بوک جدید برای Inference ---
# این کد را در یک سلول جدید در نوت بوک جدید خودتان اجرا کنید.

# --- 1. نصب کتابخانه های مورد نیاز ---
# مطمئن شوید که TPU در تنظیمات نوت بوک فعال است.
print("Installing necessary libraries...")
# bitsandbytes برای TPU استفاده نمی شود و حذف شده است.
!pip install -q -U transformers accelerate peft trl scipy datasets
!pip install -q -U "huggingface_hub[cli]"

# --- تنظیمات محیطی برای TPU ---
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel # LoraConfig, get_peft_model, prepare_model_for_kbit_training برای inference مستقیم لازم نیستند
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient # اضافه شده برای دسترسی به Kaggle Secrets

# تنظیمات GPU (CUDA_VISIBLE_DEVICES, PYTORCH_CUDA_ALLOC_CONF) برای TPU لازم نیستند و حذف شده اند.
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("Libraries installed and environment configured for TPU.")

# --- 2. لاگین به Hugging Face با استفاده از Kaggle Secrets ---
# این روش امن تر و توصیه شده برای دسترسی به توکن در Kaggle است.
try:
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("HF_TOKEN")
    login(token=secret_value)
    print("\nSuccessfully logged in to Hugging Face using Kaggle Secret.")
except Exception as e:
    print(f"\nError logging in to Hugging Face: {e}")
    print("Please ensure 'HF_TOKEN' is set as a Kaggle Secret with write access.")
    print("You can try manual login by uncommenting the line below.")
    # login() # uncomment this line to try manual login if secret method fails


# --- 3. تعریف مسیرها و نام مدل ---
# نام دیتاست که در مرحله 2 آپلود کردید (مثلاً 'fine-tuned-medgemma-output')
# مطمئن شوید که این نام با نامی که برای دیتاست خود انتخاب کردید مطابقت دارد.
KAGGLE_DATASET_NAME = "my-medgemma-finetuned-model" # *** این را با نام دقیق دیتاست خود جایگزین کنید ***

# مسیر کامل به پوشه مدل فاین تیون شده در دیتاست
# فرض بر این است که شما پوشه 'results/final_model' را دانلود کرده اید.
model_path_in_dataset = f"/kaggle/input/farsi-tuned"

# نام مدل پایه (باید با مدلی که فاین تیون کردید مطابقت داشته باشد)
base_model_name = "google/medgemma-4b-it"

print(f"\nAttempting to load fine-tuned model from: {model_path_in_dataset}")

# --- 4. بارگذاری توکنایزر ---
# توکنایزر فاین تیون شده را از مسیری که ذخیره کرده اید بارگذاری کنید.
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path_in_dataset, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("Tokenizer loaded successfully from fine-tuned path.")
except Exception as e:
    print(f"Error loading tokenizer from fine-tuned path: {e}. Falling back to base model tokenizer.")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"


# --- 5. بارگذاری مدل پایه ---
# برای TPU، مدل را بدون BitsAndBytesConfig و با bfloat16 بارگذاری می کنیم.
print(f"Loading base model '{base_model_name}' for TPU...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16, # *** اصلاح شده: استفاده از bfloat16 برای TPU ***
    device_map="auto", # برای توزیع مدل روی دستگاه های موجود (TPU cores)
)
# prepare_model_for_kbit_training برای TPU لازم نیست و حذف شده است.
print("Base model loaded for TPU.")


# --- 6. اعمال آداپتورهای LoRA فاین تیون شده ---
print(f"Applying LoRA adapters from {model_path_in_dataset} to the base model...")
model = PeftModel.from_pretrained(base_model, model_path_in_dataset)
print("LoRA adapters applied successfully.")

# --- تنظیم مدل در حالت ارزیابی ---
model.eval()
print("Model set to evaluation mode.")

# --- 7. (اختیاری) ادغام مدل LoRA با مدل پایه ---
# این مرحله مدل فاین تیون شده (LoRA) را با مدل پایه ادغام می کند
# تا یک مدل واحد و قابل استقرار ایجاد شود. این نیاز به RAM بیشتری دارد.
# اگر قصد استقرار مدل کامل را دارید، این خط را فعال کنید.
# model = model.merge_and_unload()
# print("LoRA adapters merged with base model.")


# --- 8. ایجاد Pipeline برای Inference ---
print("\nCreating text generation pipeline for inference...")
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16, # *** اصلاح شده: استفاده از bfloat16 برای Inference در TPU ***
    device_map="auto",
)

print("\nFine-tuned model ready for inference!")

# --- 9. تست مدل ---
prompt_test = "سوال: علائم آنفولانزا چیست؟\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")


Installing necessary libraries...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Libraries installed and environment configured for TPU.

Successfully logged in to Hugging Face using Kaggle Secret.

Attempting to load fine-tuned model from: /kaggle/input/farsi-tuned
Tokenizer loaded successfully from fine-tuned path.
Loading base model 'google/medgemma-4b-it' for TPU...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.77it/s]


Base model loaded for TPU.
Applying LoRA adapters from /kaggle/input/farsi-tuned to the base model...
LoRA adapters applied successfully.
Model set to evaluation mode.

Creating text generation pipeline for inference...


Device set to use cpu



Fine-tuned model ready for inference!

--- Test Question ---
سوال: علائم آنفولانزا چیست؟

Generated Answer: علائم آنفولانزا می‌تواند متفاوت باشد، اما معمولاً شامل موارد زیر است:

*   **تب:** تب معمولاً در عرض 2 تا 3 روز پس از ابتلا به آنفولانزا ظاهر می‌شود.
*   **سرفه:** سرفه معمولاً خشک است.
*   **درد بدن:** درد بدن، به خصوص در عضلات، رایج است.
*   **ضعف و خستگی:** احساس خستگی و ضعف
